<a href="https://colab.research.google.com/github/HandanYU/Rumour-detection/blob/main/Pre_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- replace_abbreviations
- remove url 
- get the emoji and replace them as words
- remove stop words
- keep only english letters
- stemming

In [ ]:
!pip install emoji
!pip install nltk

     |████████████████████████████████| 175 kB 5.4 MB/s eta 0:00:01
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=d92ce388b522979dfdc5fd450084cc9205015adc90439c093c847eca5a54a52f
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji


In [ ]:
import emoji
from nltk.corpus import stopwords
import re
import nltk
import json
import pandas as pd

In [ ]:

nltk.download('wordnet')
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
stopword = stopwords.words('english')
def clean_tweet(content):
    # replace_abbreviations
    content = content.lower()
    content = re.sub(r"won\'t", "will not", content)
    content = re.sub(r"can\'t", "can not", content)
    content = re.sub(r"cannot", "can not", content)
    content = re.sub(r"n\'t", " not", content)
    content = re.sub(r"\'re", " are", content)
    content = re.sub(r"\'s", " is", content)
    content = re.sub(r"\'d", " would", content)
    content = re.sub(r"\'ll", " will", content)
    content = re.sub(r"\'t", " not", content)
    content = re.sub(r"\'ve", " have", content)
    content = re.sub(r"\'m", " am", content)
    content = re.sub(r"\"", " am", content)

    content = re.sub(r'@[A-Za-z0-9_]+', '', content) # remove twitter ID
    # remove url 
    ## http, https
    content = re.sub(r'https?://[^ ]+', '', content) 
    ## www.
    content = re.sub(r'www.[^ ]+', '', content)
    # get the emoji and replace them as words
    emojis = emoji.distinct_emoji_list(content)
    for e in emojis:
        content = re.sub(e, emoji.demojize(e), content)
    content = re.sub('\w+\d+\w+', '', content) # remove the word contains numbers
    content = re.sub(r'[:_.!+-=——,$%^\.\?\\~\"\'@#$%&*<>{}\[\]()/]', ' ', content) # remove punctuation
    content = re.sub(r"\s+", " ", content) # conver multiple spaces as a single space
    content = content.strip()
    # remove stop words and keep only english letters
    content = [c for c in content.split(' ') if c not in stopword and c.isalpha()]
    # do stemming
    content = [stemmer.stem(token) for token in content]
    
    return content

In [ ]:
def clean_data(json_file):
  """
  json_file: 'train_reply.json'
  """
  with open(json_file, 'r+') as file:
      content=file.read()
  content=json.loads(content)
  df = pd.DataFrame(content)
  df = df.stack()
  df = df.unstack(0)
  df['text'] = df['text'].apply(lambda x: clean_tweet(x))
  return df
